Pergunta 1: Média de Valor Total por Mês
A seguir, vamos responder à pergunta: "Qual a média de valor total (total_amount) recebido em um mês considerando todos os yellow táxis da frota?".

Para isso, agrupamos os dados por ano e mês e aplicamos a função de agregação AVG() na coluna total_amount.

In [0]:
%sql
-- Pergunta 1: Qual a média de valor total (total_amount) recebido em um mês? 
SELECT
  YEAR(pickup_datetime) AS ano,
  MONTH(pickup_datetime) AS mes,
  AVG(total_amount) AS media_valor_total
FROM
  ifood_challenge.yellow_taxi_trips
GROUP BY
  ano, mes
ORDER BY
  ano, mes;

ano,mes,media_valor_total
2001,1,56.276666666666664
2002,12,37.1875
2003,1,85.74
2008,12,44.316153846153846
2009,1,10.45
2014,11,52.38
2022,10,69.13545454545455
2022,12,26.3292
2023,1,27.4578427555988
2023,2,27.365620957331974


Análise do Resultado: A tabela acima mostra a média de faturamento bruto mensal da frota de táxis. É possível observar a variação no valor médio das corridas ao longo dos meses analisados, o que pode indicar tendências de mercado ou sazonalidade.

Pergunta 2: Média de Passageiros por Hora em Maio
Agora, vamos responder à pergunta: "Qual a média de passageiros (passenger_count) por cada hora do dia que pegaram táxi no mês de maio considerando todos os táxis da frota?".

Nesta consulta, primeiro filtramos os dados para incluir apenas o mês de Maio de 2023. Em seguida, agrupamos por hora do dia para calcular a média de passageiros.

In [0]:
%sql
-- Pergunta 2: Qual a média de passageiros (passenger_count) por cada hora do dia no mês de maio? 
SELECT
  HOUR(pickup_datetime) AS hora_do_dia,
  AVG(passenger_count) AS media_de_passageiros
FROM
  ifood_challenge.yellow_taxi_trips
WHERE
  MONTH(pickup_datetime) = 5 AND YEAR(pickup_datetime) = 2023 -- Filtra para Maio de 2023
GROUP BY
  hora_do_dia
ORDER BY
  hora_do_dia;

hora_do_dia,media_de_passageiros
0,1.4274634212427746
1,1.4379867872044507
2,1.4552871886313288
3,1.452468956352008
4,1.4051493960584869
5,1.2842776281804693
6,1.2612818763889195
7,1.2820311563157494
8,1.2956814339935572
9,1.3120987899789809


Análise do Resultado: O resultado nos permite identificar os horários de pico na demanda de passageiros. Vemos como a média de passageiros por corrida flutua ao longo do dia, fornecendo uma informação valiosa para o planejamento e otimização da distribuição da frota de táxis.